## Problem Set - Dynamic life-cycle models of human capital accumulation with respy

In this problem set, we will work with Eckstein-Keane-Wolpin (EKW) models, a class of dynamic discrete choice models that are used to address economic questions in the realm of labor and education economics. Prominent examples of such models in economic research are Keane and Wolpin (1994, 1997, 2000) and more recently Blundell et al. (2016), Adda et al. (2017), and Eckstein et al. (2019). For the problem set, we will be working with a toy model that follows the famous story of Robinson Crusoe who is stranded on a deserted island. Our version of this story focuses on his work and leisure decisions during the time on the island, specifically the human capital he accumulates as he engages in productive activities. 

We will be using the Python package [respy](https://respy.readthedocs.io) to implement and analyze the economic model. Below, you find a summary of the economic framework of the problem set. For a detailed description of this specific model, please refer to the [introductory tutorial](https://respy.readthedocs.io/en/latest/tutorials/robinson_crusoe.html) in the **respy** documentation. An extended outline of EKW models using the example of the seminal model presented in Keane and Wolpin (1997) can be found in the [explanations](https://respy.readthedocs.io/en/latest/explanations/index.html) section of the documentation. 

The problem set consists of four exercises. In the first exercise you are asked to examine the simulated data in detail and identify components that make up the rewards in the model. The second exercise explores the modeling and policy evaluation capabilities of **respy** in the context of the intertemporal trade-off that Robinson is facing. The third exercise focuses on the discrete choice dynamic programming problem that lies at the heart of the structural model to explore the trade-off between computation time and model components. Lastly, the fourth exercise explores the numerical integration of the expected value functions to illustrate numerical components of model implementation.

In [1]:
import timeit
import respy as rp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Economic Setting: Robinson Crusoe on an island

- Robinson chooses every period $t = 0, \dots, T$ to either go fishing, $a = 0$, or spend the day in the hammock, $a = 1$.

- If Robinson chooses to go fishing, he gains one additional unit of experience in the next period. Experience starts at zero.

- The utility of a choice, $U(s_t, a_t)$, depends on the state $s_t$, which contains information on the individual's characteristics, and the chosen alternative $a_t$.

- Robinson's utility for any given option can be denoted by

$$\begin{align}
    U(s_t, a_t) = \underbrace{W(s_t, a_t)}_{wage} + \underbrace{N(s_t, a_t)}_{non-pecuniary}
\end{align}$$

**Working alternatives**

- For working alternatives like fishing, utility consists of two components, a wage and a non-pecuniary component. The wage is defined as 

$$\begin{align}
        W(s_t, a_t)  &= r_a \exp\{x^w_{at} \beta^w_a + \epsilon_{at}\}\\
    \ln(W(s_t, a_t)) &= \ln(r_a) + x^w_{at} \beta^w_a + \epsilon_{at}
\end{align}$$

- It consists of several components:
    - $r_a$ is a market rental price for skill units.
    - $x^w_{at}$ and $\beta^w_a$ are the choice- and time-dependent covariates and returns related to the wage signaled by superscript $w$. 
    - $\epsilon_{at}$ is a choice-specific random shock from the shock vector $\epsilon_t \sim \mathcal{N}(0, \Sigma)$ for all choices.

- The non-pecuniary rewards for working alternatives are a vector dot product of covariates $x_t^w$ and parameters $\beta^w$. The superscript $w$ signals that the components belong to working alternatives.

$$\begin{align}
    N^w(s_t, a_t) = x_t^w\beta^w_{a}
\end{align}$$

**Non-working alternatives**

- For non-working alternatives like the hammock, $W(s_t, a_t) = 0$. The non-pecuniary reward for non-working alternatives is very similar to the working alternative except that the shocks enter the equation additively. Superscript $n$ stands for non-pecuniary.

$$
    N^n(s_t, a_t) = x_t^n\beta^n_{a} + \epsilon_{at}
$$


- Robinson's choice set thus consists of a "working" alternative which awards him a pecuniary compensation or wage and a "leisure" or non-working alternative which he derives a utility from, but no experience or wage. Experience in this basic model starts at zero and increases by one for every period $t$ in $1, ..., T$ where he chooses to go fishing. 


**Robinson's choice problem**

- During his time on the island, Robinson has to make choices about which actions to take. The general assumption is that Robinson is forward-looking and maximizes the expected present value of utility over the remaining lifetime which is achieved by selecting the optimal sequence of choices $\{a_t\}^T_{t = 0}$.


- Robinson's decision problem can be expressed as a Bellman equation where the value of a given state consists of the current and future state values associated with it.  

$$\begin{align}
    V(s_{t})&= \max_{a_t} \, \{\underbrace{U(s_t, a_t)}_{\text{flow utility}}+ \delta \underbrace{\text{E}[ V(s_{t+1})]\}}_{\text{continuation value}}
\end{align}$$

- For each period, the value of a given choice consists of its flow utility and its continuation value. Since the realization of shocks becomes known in each period before Robinson makes his choice, the flow utility can be extracted from the expectation operator. As shocks in period $t + 1$ are unknown to the individual in period $t$, utility must be maximized given the joint distribution of shocks in period $t + 1$ which is a maximization problem over a two-dimensional integral. Denote the non-stochastic part of a state as $s^-$. Then, Robinson maximizes

$$\begin{equation}
    V(s_t) = \max_{a_t}\{
        U(s_t, a_t) + \delta \int_{\epsilon_{1, t + 1}} \int_{\epsilon_{0, t + 1}}
        \max_{a_{t + 1}} V_{a_{t + 1}}(s^-_{t + 1}, \epsilon_{t + 1})
        f_\epsilon(\epsilon_{t + 1})
        d_{\epsilon_{0, t + 1}}, d_{\epsilon_{1, t + 1}}
    \}
\end{equation}$$

### Setup: Loading the model

Load the example model `robinson_crusoe_basic` into memory using the function `get_example_model` by executing the cell below. The function returns a DataFrame of `params` which contains the model parameters, the dictionary `options` holding model options, and a set of simulated data.

In [2]:
params, options, data = rp.get_example_model("robinson_crusoe_basic")

The parameters correspond to the model equations. Since this is a very simple model, there are only 7 parameters. The table below shows the parameters that appear in the reward functions.

|parameter              | category          | name          | value |
|-----------------------|-------------------|---------------|-------|
|$\delta$               | **delta**         | **delta**     |  0.95 |
|$\beta^{w}_{fishing1}$ | **wage_fishing**  |**exp_fishing**|  0.30 |
|$\beta^{w}_{fishing2}$ | **nonpec_fishing**| **constant**  | -0.20 |
|$\beta^{n}_{hammock}$  | **nonpec_hammock**| **constant**  |  2.00 |

Aside from the parameters, the `params` DataFrame also contains components of the shock matrix of the model. The parameters listed under the category `shocks_sdcorr` correspond to the lower triangular of the shock variance-covariance matrix which is a square matrix with dimensions that correspond to the number of choices of the model. We can define these shock components in different ways, but here they are defined as standard deviations (diagonal elements of the matrix) and correlations of shocks between choices (lower-triangular elements).

---

### Exercise 1: Explore the data

Familiarize yourself with the `data` for our simple Robinson Crusoe model.

1. How is the data structured? What purpose do the two index levels in the DataFrame serve?
2. Inspect the data columns. What information is provided? How do the data columns correspond to the model equations? Where can you find Robinson's decisions and rewards?
3. Plot the choice frequencies for each period. What proportion of Robinsons goes fishing, how many individuals choose to relax in the hammock?
4. How much experience does Robinson accumulate over time? Plot the frequencies of experience levels for each period.

*Hint: You can use the pandas functions [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) and [pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) to compute the quantities of interest.*

---

### Exercise 2: Model mechanisms and economic analysis

Now that we have familiarized ourselves with the data, we can analyze Robinson's choices in more detail. In our dynamic model, Robinson is forward-looking and thus takes future states into account when making his decisions.

1. For now, focus on Robinson in the first period captured in the `data`. How would his choices change if he only took his current period utility (i.e. the **flow utility of period 0**) into account and ignored future rewards? Compute the choice frequencies for this scenario and compare them to his actual choices. 

2. Let's assess the choices of a myopic Robinson over all time periods. **Edit the parameter vector** accordingly and **simulate** data for this scenario. How do Robinson's choices now compare to the baseline scenario? What happens if Robinson values future rewards exactly the same as current period rewards?

3. We can also imagine a technology shock that augments Robinson's return to experience in fishing. Let's assume the shock increases his return by **10%**. Implement the technology shock and simulate data to compare Robinson's choices against the baseline scenario. Which part of Robinson's utility is affected by this adjustment? Can you think of real world examples for policy interventions which we could simulate with a model like this? 

---

### Exercise 3: Model computation time

An important aspect of structural econometric models like the one assessed in this problem set is the discrete choice dynamic programming (DCDP) problem which lies at the heart of the model. With increasing model components like choices, individual characteristics, and time periods the computational complexity of this problem increases as the potential states (state space) that have to be considered to determine Robinson's optimal choice path grows. In this exercise we will thus inspect the state space and computation time of the model in more detail. A **respy**'s model can be solved in two steps:

```
solve = rp.get_solve_func(params, options)
state_space = solve(params)

```

The first step constructs the state space for the model. In the second step, the DCDP problem is solved for a given parameter vector. **respy** utilizes two-step approaches like this for many functions to facilitate usage and cut down on computation time. In the following we assess how different modeling components affect the solution time of the model.

1. One impotant such component are the number of periods considered in a model. Solve the `robinson_crusoe_basic` model for **different numbers of periods in the model** (maximum of 30-40) and log the solution time. Repeat the exercise 10 times for each number of periods and plot the mean solve time to compare how to solution time increases with the number of periods considered in the model. **Note that you only need to time the second step ```state_space = solve(params)``` shown above**.

2. Let's now compare the solution time of **multiple models**. Assess the solution time of the models `robinson_crusoe_basic`,`robinson_crusoe_extended`, `kw_94_one`, and `kw_97_basic` (if possible) which exhibit an increasing number of choices available in the model. Set the number of periods to 20 for all models to make them more comparable in regards to time horizon and analyze the results. How many choices are available in each model and how does the solution time increase? 

    Note that there are some other components that distinguish the models than just the number of periods and available choices. Assess the `params` and `options` of the models. Which components likely also influence the solution time?
    

---

### Exercise 4: Numerical integration

One important component of the solution to the DCDP problem in **respy** models is numerical integration. Solving for the optimal decision sequence requires us to solve a multi-dimensional integral at every point in the state space to find the expected value functions. The integrated value function does not have an analytical solution and thus requires the application of numerical methods. 

The **respy** interface provides users with multiple methods for numerical integration. The relevant options to calibrate the integration method are `options["solution_draws"]` which determines the number of integration points and `options["monte_carlo_sequence"]` which determines the sequence that generates these points.

**respy** offers Monte Carlo and Quasi-Monte Carlo methods to generate sequences for integration:

- Monte Carlo Simulation: Chooses points randomly in the domain (`random`).
- Quasi Monte Carlo Simulation: Chooses points from one of the two low-discrepancy sequences (`sobol`, `halton`).


In this exercise, we seek to implement a small simulation exercise to explore numerical integration in our Robinson Crusoe example.

1. Load the `robinson_crusoe_extended` into memory and change the `solution_draws` in the `options` to a large number (e.g. 10,000) and simulate data. Compute choice frequencies based on this data. In the next steps we will treat these as the true solution and benchmark how different methods perform in comparison.
3. Now simulate data for **different numbers of solution draws**. Compute the **root-mean-square-error (RMSE) of the choice frequencies** in this data compared to the *true* solution. Repeat the exercise for the three different sequences that are available in the **respy** interface: `sobol`, `halton`, and `random` and plot the resulting RMSE against the number of solution draws.


---

### References

- Adda, J., Dustmann, C., & Stevens, K. (2017). The career costs of children. *Journal of Political Economy*, 125(2), 293-337.

- Blundell, R., Costa Dias, M., Meghir, C., & Shaw, J. (2016). Female labor supply, human capital, and welfare reform. *Econometrica*, 84(5), 1705-1753.

- Eckstein, Z., Keane, M., & Lifshitz, O. (2019). Career and family decisions: Cohorts born 1935–1975. *Econometrica*, 87(1), 217-253.

- Keane, M. P., & Wolpin, K. I. (1994). The solution and estimation of discrete choice dynamic programming models by simulation and interpolation: Monte Carlo evidence. The review of economics and statistics, 648-672.

- Keane, M. P., & Wolpin, K. I. (1997). The career decisions of young men. *Journal of Political Economy*, 105(3), 473-522.

- Keane, M. P., & Wolpin, K. I. (2000). Eliminating race differences in school attainment and labor market success. *Journal of Labor Economics*, 18(4), 614-652.